In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 02:37:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('/home/louisekuo/notebooks/data/pq/green/*/*')

In [5]:
df_green.createOrReplaceTempView('green')

In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS grass_hour 
    , PULocationID AS zone 
    , sum(total_amount) AS amount
    , count(1) AS number_records
FROM green 
WHERE date(lpep_pickup_datetime) >= date'2020-01-01'
GROUP BY 1,2 
""")

In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('/home/louisekuo/notebooks/data/report/revenue/green', mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('/home/louisekuo/notebooks/data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [15]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS grass_hour 
    , PULocationID AS zone 
    , sum(total_amount) AS amount
    , count(1) AS number_records
FROM yellow 
WHERE date(tpep_pickup_datetime) >= date'2020-01-01'
GROUP BY 1,2 
""")

In [16]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('/home/louisekuo/notebooks/data/report/revenue/yellow', mode='overwrite')

> Try read it directly from the folder

In [18]:
df_green_revenue = spark.read.parquet('/home/louisekuo/notebooks/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('/home/louisekuo/notebooks/data/report/revenue/yellow')

In [19]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [22]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['grass_hour', 'zone'], how='outer')

In [23]:
df_join.write.parquet('/home/louisekuo/notebooks/data/report/revenue/total', mode='overwrite')

> Try read it directly from the folder

In [24]:
df_join = spark.read.parquet('/home/louisekuo/notebooks/data/report/revenue/total')
df_join

DataFrame[grass_hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [28]:
df_zones = spark.read.parquet('/home/louisekuo/notebooks/zones')
df_zones

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [31]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')